<a href="https://colab.research.google.com/github/felipeEddy/Alura-IA/blob/main/Project_Alura_IA_Immersion_Star_Trek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Command to install Google AI library

In [35]:
pip install -q -U google-generativeai

Imports needed in the code

In [36]:
import pandas as pd
import numpy as np
import google.generativeai as genai

Google AI configuration

In [37]:
GOOGLE_API_KEY='YOUR_API_KEY'
genai.configure(api_key=GOOGLE_API_KEY)

model_embed = "models/embedding-001"
model_gen = "gemini-1.0-pro"

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5
}

Google AI instances

In [38]:
model_2 = genai.GenerativeModel(model_gen,generation_config=generation_config)
chat = model_2.start_chat(history=[])

Template dataframe creation

In [39]:
data = {
    'number': list(range(1, 28)),
    'title': [
        "The Man Trap", "Charlie X", "Where No Man Has Gone Before", "The Naked Time", "The Enemy Within",
        "Mudd's Women", "What Are Little Girls Made Of?", "Miri", "Dagger of the Mind", "The Corbomite Maneuver",
        "The Menagerie, Part I", "The Menagerie, Part II", "The Conscience of the King", "Balance of Terror",
        "Shore Leave", "The Galileo Seven", "Squirm", "Arena", "Tomorrow Is Yesterday",
        "The Return of the Archons", "A Taste of Armageddon", "Space Seed", "This Side of Paradise",
        "The Devil in the Dark", "Errand of Mercy", "The Alternative Factor", "The City on the Edge of Forever"
    ],
    'content': [
        "The USS Enterprise answers a distress call from an outpost on planet M-113, only to find a shape-shifting creature that feeds on salt.",
        "A troubled teenager with extraordinary powers is brought aboard the Enterprise.",
        "The Enterprise encounters a barrier at the edge of the galaxy, and its crew is forever altered.",
        "A virus strips away the crew's inhibitions, revealing their deepest desires.",
        "A transporter malfunction creates a duplicate of Kirk, splitting his personality into good and evil.",
        "Kirk encounters Harry Mudd and three beautiful women who are not what they seem.",
        "The Enterprise encounters a powerful android who claims to be the creator of humanity.",
        "The landing party is trapped on a planet where everyone over the age of puberty dies.",
        "Kirk and Spock investigate a rehabilitation colony where a new therapy is causing violent insanity.",
        "The Enterprise encounters a seemingly invincible alien vessel.",
        "Spock is put on trial for mutiny and reveals a shocking secret about Captain Pike.",
        "The story of Captain Pike continues as Spock's plan unfolds.",
        "Kirk investigates a Shakespearean actor with a dark past.",
        "The Enterprise faces off against a Romulan Bird-of-Prey in a tense game of cat-and-mouse.",
        "The crew takes shore leave on a planet that can manifest their fantasies.",
        "Spock must lead a landing party to safety after their shuttle crashes on a hostile planet.",
        "The Enterprise encounters a giant, mutated silicon-based creature.",
        "Kirk is forced to fight a Gorn captain to the death on a desolate planet.",
        "The Enterprise travels back in time to 1969.",
        "Kirk and Spock encounter a society controlled by a powerful entity called Landru.",
        "Kirk must stop two planets from destroying each other with simulated war.",
        "Khan Noonien Singh, a genetically engineered superhuman, awakens from suspended animation.",
        "The Enterprise crew is tempted by a life of idyllic pleasure on a planet of telepathic beings.",
        "Kirk must communicate with a creature that is killing miners on a mining colony.",
        "Kirk and Spock get caught in a war between the Klingon Empire and the Federation.",
        "The Enterprise encounters a being with the power to shift between parallel universes.",
        "Kirk and Spock travel back in time to prevent a tragedy that could alter history.",
    ]
}

startrek_df = pd.DataFrame(data)

Base embedding functions

In [41]:
def embed_fn(title, content):
  return genai.embed_content(model=model_embed,
                             content=content,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

def add_embed_to_df(df):
    df["embed"] = df.apply(lambda row: embed_fn(row["title"], row["content"]), axis=1)

Main search function. It is possible to use multiple dataframes for the search. In the current code I'm using only one.

In [47]:
def generate_search(query, dataframes):
    query_embed = embed_fn("", query)
    best_result = None
    best_score = -1

    for df in dataframes:
        for col in df.columns:
            if col != "embed":
                dot_products = np.dot(np.stack(df["embed"]), query_embed)
                index = np.argmax(dot_products)
                score = dot_products[index]

                if score > best_score:
                    best_score = score
                    best_result = {
                        "json": df.iloc[index].to_json(),
                        "content": df.iloc[index][col]
                    }

    return best_result

Function that will handle the conversation and final response to the user. With the Google AI chat function, it is possible to simulate a conversation. Otherwise it would return fixed text found in the dataframe.

In [43]:
def final_response(query, result):
    prompt = f"Theme: Star trek. Keep the context in as in the response. Considering this query:'{query}' and this dataframe column as response: '{result['json']}', elaborate a more user friendly response."

    response = chat.send_message(prompt)
    return response.text

User interaction code. Here the user can make questions like "What happened in planet M-113?".

In [48]:
add_embed_to_df(startrek_df)

while True:
    query = input("Enter your query (or 'exit' to close): ")
    if query.lower() == "exit":
        break

    query_resp = generate_search(query, [startrek_df])
    final_resp = final_response(query, query_resp)
    print(f"{final_resp}")

Enter your query (or 'exit' to close): What happened in planet M-113
**On planet M-113, the USS Enterprise encountered a shape-shifting creature that fed on salt.**

The creature, which could take on the form of any living being, had been terrorizing the planet's inhabitants. The Enterprise crew quickly realized that the creature was vulnerable to salt, and they used this knowledge to defeat it.

The creature was eventually killed when Captain Kirk used a phaser to vaporize it. The Enterprise crew then left M-113, leaving the planet's inhabitants to rebuild their lives.


KeyboardInterrupt: Interrupted by user